In [27]:
# import dependencies

import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from sklearn.metrics import mean_squared_error, mean_absolute_error

import pickle


In [28]:
# read the data
df = pd.read_csv('marketing_campaign_dataset_training_20K.csv')

# filter for only first 20,000 rows
filtered_df = df.iloc[:20000]

filtered_df



/var/folders/f7/dl42drj92qg78wp87m730h3m0000gn/T/ipykernel_28025/759870930.py:2: DtypeWarning: Columns (1,2,3,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('marketing_campaign_dataset_training_20K.csv')


,id,name,start_date,end_date,duration,customer_segment,target_audience,impressions,clicks,spend_email,...,spend_website,spend_youtube,spend_total,revenue_email,revenue_facebook,revenue_google_ads,revenue_instagram,revenue_website,revenue_youtube,revenue_total
0,1.0,Innovate Industries,1/1/2023,1/30/2023,30.0,Health & Wellness,Men 18-24,1922.0,506.0,2722.0,...,2694.0,2695.0,16174.0,6805.0,17260.0,14901.0,12140.0,20207.0,6739.0,78051.0
1,2.0,NexGen Systems,1/2/2023,3/2/2023,60.0,Fashionistas,Women 35-44,7523.0,116.0,1946.0,...,1927.0,1928.0,11566.0,4866.0,12343.0,10656.0,8681.0,14450.0,4819.0,55814.0
2,3.0,Alpha Innovations,1/3/2023,2/1/2023,30.0,Outdoor Adventurers,Men 25-34,7698.0,584.0,1717.0,...,1699.0,1700.0,10200.0,4291.0,10885.0,9397.0,7656.0,12743.0,4250.0,49222.0
3,4.0,DataTech Solutions,1/4/2023,3/4/2023,60.0,Health & Wellness,All Ages,1820.0,217.0,2141.0,...,2120.0,2120.0,12724.0,5353.0,13579.0,11722.0,9550.0,15897.0,5301.0,61402.0
4,5.0,NexGen Systems,1/5/2023,1/19/2023,15.0,Health & Wellness,Men 25-34,4201.0,379.0,2769.0,...,2741.0,2742.0,16452.0,6922.0,17557.0,15157.0,12348.0,20554.0,6854.0,79393.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19996.0,Innovate Industries,10/13/2023,11/11/2023,30.0,Health & Wellness,Men 18-24,5416.0,677.0,1567.0,...,1551.0,1552.0,9313.0,3918.0,9939.0,8580.0,6990.0,11635.0,3880.0,44942.0
19996,19997.0,NexGen Systems,10/14/2023,10/28/2023,15.0,Health & Wellness,Men 18-24,4376.0,338.0,992.0,...,982.0,982.0,5895.0,2480.0,6291.0,5431.0,4425.0,7365.0,2456.0,28448.0
19997,19998.0,Innovate Industries,10/15/2023,11/28/2023,45.0,Foodies,Women 35-44,6292.0,963.0,1867.0,...,1848.0,1849.0,11095.0,4668.0,11840.0,10222.0,8327.0,13861.0,4623.0,53541.0
19998,19999.0,Alpha Innovations,10/16/2023,12/14/2023,60.0,Foodies,Men 25-34,6267.0,146.0,2393.0,...,2369.0,2370.0,14220.0,5983.0,15175.0,13101.0,10673.0,17766.0,5925.0,68622.0


In [29]:
# define dependent variables

y = filtered_df[['revenue_email', 'revenue_facebook', 'revenue_google_ads', 'revenue_instagram', 
        'revenue_website', 'revenue_youtube']]

In [30]:
# # Convert Acquisition_Cost column from $ to float
# filtered_df['Acquisition_Cost'] = filtered_df['Acquisition_Cost'].str.replace('$', '').str.replace(',', '').astype(float)
# # Split the string on space and take the first part (the number), then convert to integer
# filtered_df['Duration'] = filtered_df['Duration'].str.split().str[0].astype(int)


In [31]:
# define the features (independent variables) as X
X = filtered_df.drop(['revenue_email', 'revenue_facebook', 'revenue_google_ads', 'revenue_instagram', 
        'revenue_website', 'revenue_youtube', 'revenue_total','impressions','clicks','id','name', 'spend_total', 'end_date'], axis=1)

X

,start_date,duration,customer_segment,target_audience,spend_email,spend_facebook,spend_google_ads,spend_instagram,spend_website,spend_youtube
0,1/1/2023,30.0,Health & Wellness,Men 18-24,2722.0,2655.0,2709.0,2698.0,2694.0,2695.0
1,1/2/2023,60.0,Fashionistas,Women 35-44,1946.0,1899.0,1937.0,1929.0,1927.0,1928.0
2,1/3/2023,30.0,Outdoor Adventurers,Men 25-34,1717.0,1675.0,1709.0,1701.0,1699.0,1700.0
3,1/4/2023,60.0,Health & Wellness,All Ages,2141.0,2089.0,2131.0,2122.0,2120.0,2120.0
4,1/5/2023,15.0,Health & Wellness,Men 25-34,2769.0,2701.0,2756.0,2744.0,2741.0,2742.0
...,...,...,...,...,...,...,...,...,...,...
19995,10/13/2023,30.0,Health & Wellness,Men 18-24,1567.0,1529.0,1560.0,1553.0,1551.0,1552.0
19996,10/14/2023,15.0,Health & Wellness,Men 18-24,992.0,968.0,987.0,983.0,982.0,982.0
19997,10/15/2023,45.0,Foodies,Women 35-44,1867.0,1822.0,1858.0,1851.0,1848.0,1849.0
19998,10/16/2023,60.0,Foodies,Men 25-34,2393.0,2335.0,2382.0,2372.0,2369.0,2370.0


In [32]:
# Handle missing values with mean imputation for numerical variables and mode imputation for categorical variables

# for col in X.columns:
#     if X[col].dtype == 'object':
#         X[col].fillna(X[col].mode()[0], inplace=True)
#     else:
#         X[col].fillna(X[col].mean(), inplace=True)

In [33]:
filtered_df.columns

Index(['id', 'name', 'start_date', 'end_date', 'duration', 'customer_segment',
       'target_audience', 'impressions', 'clicks', 'spend_email',
       'spend_facebook', 'spend_google_ads', 'spend_instagram',
       'spend_website', 'spend_youtube', 'spend_total', 'revenue_email',
       'revenue_facebook', 'revenue_google_ads', 'revenue_instagram',
       'revenue_website', 'revenue_youtube', 'revenue_total'],
      dtype='object')

In [34]:
# convert date column to datetime format

X['start_date'] = pd.to_datetime(X['start_date'])
X['year'] = X['start_date'].dt.year
X['month'] = X['start_date'].dt.month
X['day'] = X['start_date'].dt.day

# Drop the original date column
X = X.drop('start_date', axis=1)

# convert categorical variables to dummy variables
X = pd.get_dummies(X)

X

,duration,spend_email,spend_facebook,spend_google_ads,spend_instagram,spend_website,spend_youtube,year,month,day,customer_segment_Fashionistas,customer_segment_Foodies,customer_segment_Health & Wellness,customer_segment_Outdoor Adventurers,customer_segment_Tech Enthusiasts,target_audience_All Ages,target_audience_Men 18-24,target_audience_Men 25-34,target_audience_Women 25-34,target_audience_Women 35-44
0,30.0,2722.0,2655.0,2709.0,2698.0,2694.0,2695.0,2023,1,1,False,False,True,False,False,False,True,False,False,False
1,60.0,1946.0,1899.0,1937.0,1929.0,1927.0,1928.0,2023,1,2,True,False,False,False,False,False,False,False,False,True
2,30.0,1717.0,1675.0,1709.0,1701.0,1699.0,1700.0,2023,1,3,False,False,False,True,False,False,False,True,False,False
3,60.0,2141.0,2089.0,2131.0,2122.0,2120.0,2120.0,2023,1,4,False,False,True,False,False,True,False,False,False,False
4,15.0,2769.0,2701.0,2756.0,2744.0,2741.0,2742.0,2023,1,5,False,False,True,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,30.0,1567.0,1529.0,1560.0,1553.0,1551.0,1552.0,2023,10,13,False,False,True,False,False,False,True,False,False,False
19996,15.0,992.0,968.0,987.0,983.0,982.0,982.0,2023,10,14,False,False,True,False,False,False,True,False,False,False
19997,45.0,1867.0,1822.0,1858.0,1851.0,1848.0,1849.0,2023,10,15,False,True,False,False,False,False,False,False,False,True
19998,60.0,2393.0,2335.0,2382.0,2372.0,2369.0,2370.0,2023,10,16,False,True,False,False,False,False,False,True,False,False


In [35]:
# Split the data into training and testing sets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


In [36]:
X_train

,duration,spend_email,spend_facebook,spend_google_ads,spend_instagram,spend_website,spend_youtube,year,month,day,customer_segment_Fashionistas,customer_segment_Foodies,customer_segment_Health & Wellness,customer_segment_Outdoor Adventurers,customer_segment_Tech Enthusiasts,target_audience_All Ages,target_audience_Men 18-24,target_audience_Men 25-34,target_audience_Women 25-34,target_audience_Women 35-44
9484,30.0,3345.0,3263.0,3329.0,3315.0,3311.0,3312.0,2023,12,26,True,False,False,False,False,False,False,False,True,False
839,30.0,1438.0,1403.0,1431.0,1425.0,1423.0,1424.0,2023,4,20,False,True,False,False,False,False,False,False,False,True
8050,45.0,3208.0,3130.0,3193.0,3180.0,3176.0,3177.0,2023,1,21,True,False,False,False,False,False,False,False,True,False
9343,45.0,1345.0,1313.0,1339.0,1333.0,1332.0,1332.0,2023,8,7,False,False,False,True,False,False,True,False,False,False
2331,60.0,2618.0,2554.0,2605.0,2594.0,2591.0,2592.0,2023,5,22,False,True,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15565,45.0,1412.0,1378.0,1406.0,1400.0,1398.0,1398.0,2023,8,24,False,False,False,True,False,False,False,False,True,False
8853,30.0,866.0,845.0,862.0,858.0,857.0,858.0,2023,4,4,False,False,False,True,False,False,False,False,False,True
18734,15.0,2011.0,1962.0,2002.0,1993.0,1991.0,1992.0,2023,4,30,False,False,False,True,False,False,False,True,False,False
16298,15.0,1143.0,1115.0,1137.0,1133.0,1131.0,1132.0,2023,8,27,False,True,False,False,False,False,False,False,True,False


In [37]:
# # Remove the dollar sign and commas, then convert to float
# y_train = y_train.str.replace('$', '').str.replace(',', '').astype(float)
# y_test = y_test.str.replace('$', '').str.replace(',', '').astype(float)

In [38]:
# instantiate models
lr = MultiOutputRegressor(LinearRegression())
rf = MultiOutputRegressor(RandomForestRegressor(random_state=42, n_estimators=100, verbose=1))

In [39]:
# fit the models
lrModel = lr.fit(X_train, y_train)
rfModel = rf.fit(X_train, y_train)


[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    2.2s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    2.2s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    2.2s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    2.2s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    2.2s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    2.1s


In [40]:
# evaluate the model on the test data
print(f"lr_score: {lrModel.score(X_test, y_test)}")
print(f"rf_score: {rfModel.score(X_test, y_test)}")

lr_score: 0.9999999617974957


[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s


rf_score: 0.9999999499765151


[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s


In [41]:
# do predictions on the test data
pred_lr = lrModel.predict(X_test)
pred_rf = rfModel.predict(X_test)
pred_avg = (pred_lr + pred_rf) / 2


[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s


In [42]:
# calculate the performance of the random forest model

print('Linear Regression MSE:', mean_squared_error(y_test, pred_lr))
print('Linear Regression MAE:', mean_absolute_error(y_test, pred_lr))

print('Random Forest MSE:', mean_squared_error(y_test, pred_rf))
print('Random Forest MAE:', mean_absolute_error(y_test, pred_rf))

print('Combined MSE:', mean_squared_error(y_test, pred_avg))
print('Combined MAE:', mean_absolute_error(y_test, pred_avg))


Linear Regression MSE: 0.45296355034229485
Linear Regression MAE: 0.5136253719378034
Random Forest MSE: 0.6203940958333358
Random Forest MAE: 0.5511337500000121
Combined MSE: 0.44318788248192603
Combined MAE: 0.49226934443457465


In [43]:
# # get coefficients of the linear regression model
# coefficients = lrModel.coef_
coefficients = [estimator.coef_ for estimator in lr.estimators_]


# # get the feature importance of the random forest model
# importance = rfModel.feature_importances_
importance = [estimator.feature_importances_ for estimator in rf.estimators_]



In [44]:
# Create graphs for the feature importance and coefficients

# for i, coef in enumerate(coefficients):
#     plt.figure(figsize=(10, 6))  # Adjust the size as needed
#     plt.bar(range(len(coef)), coef)
#     plt.xticks(range(len(coef)), X_train.columns, rotation=90)  # Adjust X_train.columns accordingly
#     plt.title(f'Linear Regression Coefficients for Target {i}')
#     plt.show()

# columns = X_train.columns
# lrGraph = pd.Series(coefficients, columns).sort_values(ascending=False)
# rfGraph = pd.Series(importance, columns).sort_values(ascending=False)
# columns

In [45]:
# Visualize the feature importance and coefficients

# figure(figsize=(10,10))
# lrGraph.plot(kind='barh', title='Coefficients', color='green')


In [46]:
#Save the models to disk

rf_filename = 'marketing_planner_random_forest_model.sav'
lr_filename = 'marketing_planner_linear_regression_model.sav'
pickle.dump(rfModel, open(rf_filename, 'wb'))
pickle.dump(lrModel, open(lr_filename, 'wb'))
